# Task 3 — Multimodal Housing Prices

Images → CNN embeddings (ResNet18) + Tabular → Regressor.

In [ ]:
import pandas as pd, numpy as np, os
from PIL import Image
import torch
from torch import nn
from torchvision import models, transforms
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
import xgboost as xgb


## 1) Load data

In [ ]:
df = pd.read_csv('housing_tabular.csv')  # columns: features..., image_path, price
assert 'image_path' in df.columns and 'price' in df.columns


## 2) Image transform + feature extractor

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
model.fc = nn.Identity()
model.to(device).eval()

img_tf = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225])
])

def img_embed(path):
    img = Image.open(path).convert('RGB')
    t = img_tf(img).unsqueeze(0).to(device)
    with torch.no_grad():
        emb = model(t).squeeze(0).cpu().numpy()
    return emb


## 3) Build embedding matrix

In [ ]:
embs = []
for p in tqdm(df['image_path']):
    embs.append(img_embed(p))
embs = np.vstack(embs)


## 4) Merge with tabular features

In [ ]:
tab_cols = [c for c in df.columns if c not in ['image_path','price']]
X_tab = df[tab_cols].select_dtypes(include=[np.number]).fillna(0).values
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_tab = scaler.fit_transform(X_tab)

X = np.hstack([embs, X_tab])
y = df['price'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


## 5) Train regressor & evaluate

In [ ]:
model_xgb = xgb.XGBRegressor(n_estimators=400, learning_rate=0.05, max_depth=6, subsample=0.8, colsample_bytree=0.8, random_state=42)
model_xgb.fit(X_train, y_train)

pred = model_xgb.predict(X_test)
mae = mean_absolute_error(y_test, pred)
rmse = mean_squared_error(y_test, pred, squared=False)
mae, rmse


## 6) Save artifacts

In [ ]:
import joblib
joblib.dump({'scaler': scaler, 'xgb': model_xgb}, 'multimodal_model.joblib')
